In [1]:
import json
import pandas as pd
from sql_queries import ValorantQueries

In [2]:
vq = ValorantQueries()
vq.set_match(43621)

Connected to database valorant successfully
[('teams',), ('roundevents',), ('players',), ('events',), ('series',), ('matches',), ('maps',), ('rounds',), ('roundeconomies',), ('roundlocations',), ('playermapinstance',), ('assists',)]


In [3]:
vq.aggregate_gamestate(5)

,Timestamp,EventType,ATK_Loadout_Value,ATK_Spread,ATK_Shield,ATK_Weapon_Price,ATK_Initiator,ATK_Duelist,ATK_Controller,Map_Name,...,ATK_Def_has_OP,DEF_Spread,DEF_Loadout_Value,DEF_Shield,DEF_Weapon_Price,DEF_Initiator,DEF_Duelist,DEF_Controller,DEF_Sentinel,DEF_Def_has_OP
0,0,start,24000,0.000000,250,14500,1,2,1,Icebox,...,0,0.000000,23250,200,16600,2,1,1,1,1
1,72891,kill,24000,190.295863,250,14500,1,2,1,Icebox,...,0,399.811147,18550,150,13700,2,1,1,0,1
2,73216,kill,18500,192.810376,200,11600,1,2,1,Icebox,...,0,430.281710,18550,150,13700,2,1,1,0,1
3,80084,kill,13900,211.463987,150,8700,0,2,1,Icebox,...,0,272.688024,18550,150,13700,2,1,1,0,1
4,80351,kill,13900,257.665641,150,8700,0,2,1,Icebox,...,0,270.320132,12900,125,8700,2,0,1,0,0
5,86246,kill,13900,196.345859,150,8700,0,2,1,Icebox,...,0,252.810508,8300,75,5800,1,0,1,0,0
6,88979,kill,13900,103.062265,150,8700,0,2,1,Icebox,...,0,322.981209,3700,25,2900,0,0,1,0,0
7,92160,plant,13900,132.320204,150,8700,0,2,1,Icebox,...,0,0.000000,3700,25,2900,0,0,1,0,0
8,95954,kill,13900,175.595798,150,8700,0,2,1,Icebox,...,0,0.000000,0,0,0,0,0,0,0,0


### AAA